# Age and Biomass

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:




In [1]:
import ee
import geemap
from utils import *
initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year

mapbiomas, lulc = desired_lulc()


## Biomass - Export secondary

Biomass data is in hectares, but mapbiomas data is 30m resolution.

To deal with edge pixels, we aggregate biomass values to 30m by using the mean (so there is a buffer for land use pixels caught in between two biomass values).

## Remove isolated pixels

In the map, there were isolated pixels, often around the edges of forest patches. These would likely be due to misclassification, or follow different behaviors due to edge effects.

To avoid this issue, a kernel is applied here to include only secondary forest patches that are mostly surrounded by other secondary forest pixels.

In [ ]:
def export_secondary(age, name):

    # select the data points that match their classification as young or old by IPCC
    filter1 = ee.Image("NASA/ORNL/global_forest_classification_2020/V1/00N_10N")
    filter2 = ee.Image("NASA/ORNL/global_forest_classification_2020/V1/10N_20N")
    filter_ages = ee.ImageCollection([filter1, filter2]).mosaic()
    filter_young_secondary = filter_ages.eq(2)
    filter_old_secondary = filter_ages.eq(3)
    young_secondary = age.lte(20).updateMask(filter_young_secondary).unmask(0)
    old_secondary = age.gt(20).updateMask(filter_old_secondary).unmask(0)
    ipcc_mask = young_secondary.add(old_secondary)

    # # ----------------  Select only patches of at least one hectare  ----------------
    # # Identify all connected components (patches) in the image
    # age_patches = age.connectedComponents(ee.Kernel.plus(1), 256)
    # # Calculate the area of each patch
    # patchAreas = age_patches.select("labels").selfMask().addBands(ee.Image.pixelArea()).reduceConnectedComponents(ee.Reducer.sum(), 'labels', 256)
    # # Select only patches of at least one hectare
    # largePatches = patchAreas.gte(10000).selfMask()

    # ----------------  Exclude edge pixels ----------------
    # convert non-forest pixels from NA to zero
    # check what is the most frequent value within each hectare - if it's zero, it means the pixel is surrounded by non-forest cover
    exclude_edge_mask = age.unmask(0).focalMode(kernelType = "circle", radius = 100, units = "meters").gt(0).selfMask()

    distance_to_border_mask = ee.Image(f"{data_folder}/distance_to_border_mask")

    age = age.updateMask(ipcc_mask.And(exclude_edge_mask).And(distance_to_border_mask)).rename("age")

    export_image(age, name, region = roi, scale = 30)


export_secondary(mapbiomas, f"mapbiomas_{last_year}")